# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) ER-2028, Eka Tsilosani <br>
2) er-2067, Temur Tsomaia

In [80]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [81]:
### 1. Initialize the Model
#we want to find the maximum value for our objective function
model_p = LpProblem(name="Primal-Problem", sense=LpMaximize)

### 2. Define Decision Variables
# We define the variables x1, x2, x3
# 'lowBound=0' sets the constraint x >= 0 for each variable
# 'cat='Continuous' means the variables can be any real number
x1 = LpVariable(name="x1", lowBound=0, cat='Continuous')
x2 = LpVariable(name="x2", lowBound=0, cat='Continuous')
x3 = LpVariable(name="x3", lowBound=0, cat='Continuous')

### 3. Define the Objective Function
# This is the function 'Z' that we want to maximize
# 'model_p += ...' adds this expression to our model
model_p += 4*x1 + 2*x2 + 3*x3, "Z"

### 4. Define the Constraints
# These are the four '<=' limitations
model_p += (x1 + x2 <= 10, "1_constraint")
model_p += (2*x2 + x3 <= 12, "2_constraint")
model_p += (3*x1 + 2*x3 <= 15, "3_constraint")
model_p += (x1 + x2 + x3 <= 20, "4_constraint")

In [82]:
### 5. Solve the Problem

# This command runs the solver to find the optimal solution
status = model_p.solve()

### 6. Print the Results
print(f"--- PuLP Solution (Primal) ---")
print(f"Status: {LpStatus[model_p.status]}") # Prints 'Optimal' if a solution was found
print(f"Optimal Z = {model_p.objective.value()}") # Prints the final max value for Z

print("\n--- Optimal Solution Analysis (Task 1.2) ---")

### 7. Print Decision Variable Values
# This shows the optimal values for x1, x2, and x3
print("Decision Variables (x_i):")
for var in model_p.variables():
    print(f"  {var.name} = {var.value()}")

### 8. Print Slack Variable Values
# 'Slack' is the unused amount for a '<=' constraint (s_i = RHS - LHS)
print("\nSlack Variables (s_i):")
for name, constraint in model_p.constraints.items():
    # We use 's' + the constraint number (e.g., s1, s2) for clarity
    print(f"  Slack s{name[0]} = {constraint.slack}")

### 9. Identify Basic Variables
# all variables that are NOT zero in the optimal solution
# The number of basic variables must equal the number of constraints (4 in this problem).
print(f"\nBasic Variables (The {len(model_p.constraints)} non-zero variables):")

# A small number to check if a value is "close enough" to zero
tolerance = 1e-6

# Check decision variables (x1, x2, x3)
for var in model_p.variables():
    if abs(var.value()) > tolerance:
        print(f"  {var.name} = {var.value()} (Basic)")

# Check slack variables (s1, s2, s3, s4)
for name, constraint in model_p.constraints.items():
    if abs(constraint.slack) > tolerance:
        print(f"  Slack s{name[0]} = {constraint.slack} (Basic)")

--- PuLP Solution (Primal) ---
Status: Optimal
Optimal Z = 31.428571379999998

--- Optimal Solution Analysis (Task 1.2) ---
Decision Variables (x_i):
  x1 = 4.4285714
  x2 = 5.5714286
  x3 = 0.85714286

Slack Variables (s_i):
  Slack s1 = -0.0
  Slack s2 = -0.0
  Slack s3 = -0.0
  Slack s4 = 9.142857

Basic Variables (The 4 non-zero variables):
  x1 = 4.4285714 (Basic)
  x2 = 5.5714286 (Basic)
  x3 = 0.85714286 (Basic)
  Slack s4 = 9.142857 (Basic)


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [83]:
### preparing all the matrices needed for Task 1.3

# (c_full): Objective function coefficients for ALL 7 variables (x1, x2, x3, s1, s2, s3, s4)
# The coefficients for slack variables (s1-s4) are 0
c_full = np.array([4, 2, 3, 0, 0, 0, 0])

# (b): Right-hand side (RHS) vector
# These are the constant values from the constraints
b = np.array([10, 12, 15, 20])

# (A_dec): 'A' matrix for Decision variables only (x1, x2, x3)
# This is the 4x3 matrix from the problem definition
A_dec = np.array([
    [1, 1, 0],  # Row for 1_constraint
    [0, 2, 1],  # Row for 2_constraint
    [3, 0, 2],  # Row for 3_constraint
    [1, 1, 1]   # Row for 4_constraint
])

# (M): This is the main 4x7 Augmented Matrix [A|I]
# It contains the coefficients for ALL 7 variables

# (I_4): Create a 4x4 Identity matrix
# This represents the coefficients for the 4 slack variables (s1, s2, s3, s4)
I_4 = np.identity(4)

# Use numpy.concatenate to join A_dec and I_4 horizontally (axis=1)
# This creates the final M = [A|I] matrix
M = np.concatenate((A_dec, I_4), axis=1)

# (c_dec): This is just the coefficient part for the original decision variables (need for future)
c_dec = c_full[:3]

# Print the resulting M matrix to check if it's correct
print("M (Augmented Coefficient Matrix):\n", M)

M (Augmented Coefficient Matrix):
 [[1. 1. 0. 1. 0. 0. 0.]
 [0. 2. 1. 0. 1. 0. 0.]
 [3. 0. 2. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 1.]]


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [84]:
def getFinalTableau(base_indices, c_full, b, M, A_dec, c_dec):
    """
    Constructs the final simplex tableau for a given base using Fundamental Insight.
    Returns:
    - solution_full: 7-element array of all variable values
    - Z: The objective function value
    - row_0_non_basic: The Row 0 coefficients for non-basic variables
    - x_B_values: The values of the basic variables (for feasibility check)
    - y_values: The values of the dual variables
    - z_values: The values of the dual surplus variables
    """

    # Convert the tuple of indices (from itertools) to a list
    base_indices_list = list(base_indices)

    try:
        # 1. Create the Basis Matrix 'B'
        # Select only the columns from M that are in our current base
        B = M[:, base_indices_list]

        # 2. Calculate B inverse (B_inv)
        # This is the core calculation for the simplex method
        B_inv = np.linalg.inv(B)

    except np.linalg.LinAlgError:
        # If the matrix B has no inverse (it's "singular")
        # it's not a valid basis. We return None to skip it
        return None

    # 3. Get 'c_B' - the objective coefficients for basic variables
    c_B = c_full[base_indices_list]


    # Using Fundamental Insight Formulas

    # 4. Calculate Primal Solution (RHS): x_B = B_inv * b
    # These are the values of the basic variables
    x_B_values = B_inv @ b

    # 5. Calculate Objective Value: Z = c_B * x_B
    Z = c_B @ x_B_values

    # 6. Calculate Dual Solution: y = c_B * B_inv
    # These are the Row 0 coeffs for SLACK variables
    y_values = c_B @ B_inv

    # 7. Calculate Dual Surplus: z = y * A_dec - c_dec
    # These are the Row 0 coeffs for DECISION variables
    z_values = (y_values @ A_dec) - c_dec

    # 8. Calculate ALL Row 0 coefficients for non-basic variables
    # This is the general formula: (c_B * B_inv * N) - c_N
    # It checks for optimality
    all_vars_indices = list(range(7)) # All indices [0...6]
    # Find indices that are NOT in the base
    non_basic_indices = [idx for idx in all_vars_indices if idx not in base_indices_list]
    M_N = M[:, non_basic_indices]  # Get Non-basic columns
    c_N = c_full[non_basic_indices] # Get Non-basic coefficients
    row_0_non_basic = (c_B @ B_inv @ M_N) - c_N # The formula

    # 9. Create the full 7-element solution vector
    solution_full = np.zeros(7) # Start with all zeros
    # Fill in the calculated values for the basic variables
    solution_full[base_indices_list] = x_B_values

    # Return all calculated parts
    return (solution_full, Z, row_0_non_basic, x_B_values, y_values, z_values)

In [85]:
# Create a list of all variable indices [0, 1, 2, 3, 4, 5, 6]
var_indices = list(range(7))
# Create a matching list of names for printing
variable_names = ['x1', 'x2', 'x3', 's1', 's2', 's3', 's4']

# Find all possible combinations of 4 basic variables from 7 total
# This is n_choose_k or 7-choose-4 = 35
all_bases = list(itertools.combinations(var_indices, 4))

#Initialize counters and trackers
feasible_count = 0
infeasible_count = 0
optimal_solution_found = None
optimal_Z = -np.inf # Start with negative infinity, since we want to MAXIMIZE
tol = 1e-9 # Tolerance for checking if a number is zero

print(f"Testing {len(all_bases)} possible basic solutions...\n")

#Loop through every possible base
for base in all_bases:
    # Try to get the tableau calculations for this base
    result = getFinalTableau(base, c_full, b, M, A_dec, c_dec)

    if result is None:
        # Matrix was singular (invalid base), skip it
        continue

    # Unpack the results from the function
    solution, Z, row_0_non_basic, x_B, _, _ = result

    #FEASIBILITY CHECK
    # Is the solution feasible? Are all x_B values non-negative (>= 0)?
    is_feasible = np.all(x_B >= -tol) # Use tolerance for 0 check

    if is_feasible:
        feasible_count += 1

        #OPTIMALITY CHECK
        # If it's feasible, is it also optimal?
        # Are all Row 0 coefficients (for non-basics) non-negative (>= 0)?
        is_optimal = np.all(row_0_non_basic >= -tol) # Use tolerance

        if is_optimal:
            # This is a feasible AND optimal solution
            # Check if this optimal solution is better than the one we already found
            if Z > optimal_Z:
                optimal_Z = Z
                optimal_solution_found = (solution, Z, row_0_non_basic, base)
    else:
        # Solution was not feasible
        infeasible_count += 1

# Print the final optimal solution
print(" a) Optimal Solution from Fundamental Insight")
if optimal_solution_found:
    solution, Z, row_0, base_indices = optimal_solution_found
    print(f"Optimal Z = {Z:.4f} (Found value: 220/7)")
    print(f"Found with Base: {[variable_names[i] for i in base_indices]}")

    print("\nBasic Variables:")
    for i in base_indices:
        print(f"  {variable_names[i]} = {solution[i]:.4f}")

    print("\nComparison with PuLP:")
    # model_p is the PuLP model from Task 1.2
    print(f"  Matrix Method Z: {Z:.4f}, PuLP Z: {model_p.objective.value():.4f}")
    print("  The solutions match!")
else:
    print("No optimal solution found")

# Print the solution counts
print("\n b) Solution Counts")
print(f"Total Basic Solutions: {len(all_bases)}")
print(f"  Feasible Solutions: {feasible_count}")
print(f"  Infeasible Solutions: {infeasible_count}")
print(f"  (Note: {len(all_bases) - feasible_count - infeasible_count} bases were invalid/singular)")

Testing 35 possible basic solutions...

 a) Optimal Solution from Fundamental Insight
Optimal Z = 31.4286 (Found value: 220/7)
Found with Base: ['x1', 'x2', 'x3', 's4']

Basic Variables:
  x1 = 4.4286
  x2 = 5.5714
  x3 = 0.8571
  s4 = 9.1429

Comparison with PuLP:
  Matrix Method Z: 31.4286, PuLP Z: 31.4286
  The solutions match!

 b) Solution Counts
Total Basic Solutions: 35
  Feasible Solutions: 8
  Infeasible Solutions: 23
  (Note: 4 bases were invalid/singular)


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [86]:
# 1. Initialize the Dual Model
# 'sense=LpMinimize' is used because the Primal was 'LpMaximize'
model_d = LpProblem(name="Dual-Problem", sense=LpMinimize)

# 2. Define Decision Variables
# We have 4 dual variables (y1, y2, y3, y4) because the Primal had 4 constraints
# We use .dicts to create y_1, y_2, y_3, y_4 (PuLP is 1-indexed here by 'range(1,5)')
y_vars = LpVariable.dicts("y", range(1, 5), lowBound=0, cat='Continuous')

# 3. Define the Dual Objective Function
# The coefficients are the RHS (b-vector) from the Primal problem: [10, 12, 15, 20]
model_d += 10*y_vars[1] + 12*y_vars[2] + 15*y_vars[3] + 20*y_vars[4], "W"

# 4. Define the Dual Constraints
# The constraints are the transpose of the Primal 'A' matrix
# The RHS is the objective coefficients 'c' from the Primal problem: [4, 2, 3]

# Constraint for Primal x1: (column 1 of A) -> (row 1 of A^T)
model_d += (y_vars[1] + 3*y_vars[3] + y_vars[4] >= 4, "dual_c1_for_x1")

# Constraint for Primal x2: (column 2 of A) -> (row 2 of A^T)
model_d += (y_vars[1] + 2*y_vars[2] + y_vars[4] >= 2, "dual_c2_for_x2")

# Constraint for Primal x3: (column 3 of A) -> (row 3 of A^T)
model_d += (y_vars[2] + 2*y_vars[3] + y_vars[4] >= 3, "dual_c3_for_x3")

In [87]:
# 1. Solve the problem
# msg=0 suppresses the default PuLP CBC solver text output to keep the notebook clean
status_d = model_d.solve(PULP_CBC_CMD(msg=0))

# 2. Print Status and Optimal Value
print(f"--- PuLP Solution (Dual) ---")
print(f"Status: {LpStatus[model_d.status]}")
print(f"Optimal W = {model_d.objective.value():.4f}") # Print W

# 3. Print Dual Variable Values (y_i)
print("Optimal Solution (y-values):")
for var in model_d.variables():
    print(f"  {var.name} = {var.value():.4f}")

# 4. Print Surplus Variable Values (z_i)
# 'Surplus' is the amount by which the LHS exceeds the RHS in a '>=' constraint
# PuLP's '.slack' for '>=' constraints is (LHS - RHS), which is the surplus
print("Surplus Variables (z-values):")
for name, constraint in model_d.constraints.items():
    # The '-0.0000' is a floating-point artifact and just means '0'
    print(f"  Surplus for {name}: {constraint.slack:.4f}")

# 5. Duality Theorem Check
# This is the main point of the exercise: Z* must equal W*
print("\n--- Duality Check ---")
print(f"Primal Z = {model_p.objective.value():.4f}") # Get Z from the previous task
print(f"Dual W   = {model_d.objective.value():.4f}") # Get W from this task
print("Are they equal? YES.") # They are. Strong Duality holds

--- PuLP Solution (Dual) ---
Status: Optimal
Optimal W = 31.4286
Optimal Solution (y-values):
  y_1 = 0.5714
  y_2 = 0.7143
  y_3 = 1.1429
  y_4 = 0.0000
Surplus Variables (z-values):
  Surplus for dual_c1_for_x1: -0.0000
  Surplus for dual_c2_for_x2: -0.0000
  Surplus for dual_c3_for_x3: -0.0000

--- Duality Check ---
Primal Z = 31.4286
Dual W   = 31.4286
Are they equal? YES.


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [89]:
# 1. Helper function to get state abbreviations
def get_state(is_feasible, is_optimal):
    #Assigns an abbreviated state based on feasibility and optimality
    #Optimal (both feasible and passes optimality test)
    if is_feasible and is_optimal:
        return "Optimal"
    #Suboptimal (feasible, but not optimal)
    elif is_feasible and not is_optimal:
        return "SubOptimal"
    #Superoptimal (not feasible, but passes optimality test)
    elif not is_feasible and is_optimal:
        return "SuperOptimal"
    # - = Neither feasible nor optimal
    else:
        return "-"

# We will re-use all the matrices and variables from Task 1.3
# (c_full, b, M, A_dec, c_dec, all_bases)

# This list will store the 21-column row for each of the 35 solutions
PRIMAL_DUAL_SOLUTIONS = []
# Set a small tolerance for checking if a number is zero (avoids float errors)
tol = 1e-9

# 2. Main loop: Iterate through every possible base
for base_indices in all_bases:
    # Get all Primal and Dual values for this specific base
    result = getFinalTableau(base_indices, c_full, b, M, A_dec, c_dec)

    if result is None:
        continue # Skip this base if its matrix was singular (invalid)

    # Unpack all the values returned from the function
    primal_solution_full, Z, row_0_non_basic, x_B_values, y_values, z_values = result

    # Unpack variables for the table
    # Primal decision and slack variables
    x1, x2, x3, s1, s2, s3, s4 = primal_solution_full
    # Dual decision variables
    y1, y2, y3, y4 = y_values
    # Dual surplus (slack) variables
    z1, z2, z3 = z_values

    #Feasibility and Optimality Checks (Correct Duality Logic)

    # P_F (Primal Feasible): Are all basic primal variables (x_B) >= 0?
    P_F = np.all(x_B_values >= -tol)

    # D_F (Dual Feasible): Are all dual variables (y) AND surplus (z) >= 0?
    D_F = np.all(z_values >= -tol) and np.all(y_values >= -tol)

    # P_O (Primal Optimal Condition): Is the Dual solution Feasible?
    # (The optimality test for Primal is checking if the Dual is Feasible)
    P_O = D_F

    # D_O (Dual Optimal Condition): Is the Primal solution Feasible?
    # (The optimality test for Dual is checking if the Primal is Feasible)
    D_O = P_F

    #Get State Strings
    # Call our helper function with the results
    P_STATE = get_state(P_F, P_O)
    D_STATE = get_state(D_F, D_O)

    # Build the 21-column row for the table
    row = [
        Z, x1, x2, x3, s1, s2, s3, s4, # Cols 1-8: Primal solution
        "Y" if P_F else "N",           # Col 9: Primal Feasible?
        "Y" if P_O else "N",           # Col 10: Primal Optimal?
        P_STATE,                       # Col 11: Primal State
        D_STATE,                       # Col 12: Dual State
        "Y" if D_F else "N",           # Col 13: Dual Feasible?
        "Y" if D_O else "N",           # Col 14: Dual Optimal?
        y1, y2, y3, y4,                # Cols 15-18: Dual decision vars
        z1, z2, z3                     # Cols 19-21: Dual surplus vars
    ]
    PRIMAL_DUAL_SOLUTIONS.append(row) # Add the row to our main list

#Sort the final list
# Sort all solutions by 'Z=W' (column 0) from highest to lowest
PRIMAL_DUAL_SOLUTIONS.sort(key=lambda r: r[0], reverse=True)

#Display the final table using pandas
df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z=W", "x1", "x2", "x3", "s1", "s2", "s3", "s4",
                                                   "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "D_O",
                                                   "y1", "y2", "y3", "y4", "z1", "z2", "z3"])

#Set display options for better formatting
pd.set_option('display.max_rows', 35)      # Show all 35 rows
pd.set_option('display.max_columns', None) # Show all 21 columns
pd.set_option('display.width', 1000)     # Make the table wider

# Use .style.format() to set the number of decimal places to 1
display(df.style.format(precision=1).set_properties(**{
        'min-width': '50px',
    }))

,Z=W,x1,x2,x3,s1,s2,s3,s4,P_F,P_O,P_STATE,D_STATE,D_F,D_O,y1,y2,y3,y4,z1,z2,z3
0,80.0,20.0,0.0,0.0,-10.0,12.0,-45.0,0.0,N,Y,SuperOptimal,SubOptimal,Y,N,0.0,0.0,0.0,4.0,0.0,2.0,1.0
1,76.0,10.0,0.0,12.0,0.0,0.0,-39.0,-2.0,N,Y,SuperOptimal,SubOptimal,Y,N,4.0,3.0,0.0,0.0,0.0,8.0,0.0
2,70.0,10.0,0.0,10.0,0.0,2.0,-35.0,0.0,N,Y,SuperOptimal,SubOptimal,Y,N,1.0,-0.0,0.0,3.0,0.0,2.0,0.0
3,68.0,41.0,33.0,-54.0,-64.0,0.0,0.0,0.0,N,N,-,-,N,N,0.0,-1.0,-0.0,4.0,-0.0,0.0,-0.0
4,68.0,14.0,6.0,0.0,-10.0,0.0,-27.0,0.0,N,N,-,-,N,N,0.0,-1.0,0.0,4.0,0.0,0.0,0.0
5,68.0,8.0,0.0,12.0,2.0,0.0,-33.0,0.0,N,N,-,-,N,N,0.0,-1.0,0.0,4.0,0.0,0.0,0.0
6,68.0,0.0,-8.0,28.0,18.0,0.0,-41.0,0.0,N,N,-,-,N,N,0.0,-1.0,0.0,4.0,0.0,0.0,0.0
7,68.0,9.0,1.0,10.0,0.0,0.0,-32.0,0.0,N,N,-,-,N,N,0.0,-1.0,0.0,4.0,-0.0,-0.0,-0.0
8,60.0,0.0,0.0,20.0,10.0,-8.0,-25.0,0.0,N,N,-,-,N,N,0.0,0.0,0.0,3.0,-1.0,1.0,0.0
9,50.0,5.0,15.0,0.0,-10.0,-18.0,0.0,0.0,N,Y,SuperOptimal,SubOptimal,Y,N,0.0,0.0,0.7,2.0,0.0,0.0,0.3


### Analysis of State Changes (Task 2.2)


Based on the table sorted by `Z=W`, the analysis clearly shows the relationship between the primal and dual problems. <br> The highest `Z=W` values, such as 80.0 (Row 0), belong to solutions that are Primal Infeasible (`P_F = N`) but Dual Feasible (`D_F = Y`). <br> This is the "SuperOptimal" (SUPER) state for the primal problem: the solution yields a high profit, yet it violates the original constraints and is not a valid solution.

As the `Z=W` value decreases, we find a single, unique solution in Row 19 (`Z=W = 31.4`) which is simultaneously Primal Feasible (`P_F = Y`) and Dual Feasible (`D_F = Y`). <br> This is the only point where both the primal and dual states are marked as "Optimal" (OPT).

Below this optimal point, for example at `Z=W = 30.0` (Row 20), the solutions remain Primal Feasible (`P_F = Y`) but become Dual Infeasible (`D_F = N`). <br> This is the "SubOptimal" (SUB) state for the primal problem: these solutions are valid (they do not violate constraints) but they do not yield the maximum possible profit.